In [2]:
import os
# Define Spark version
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION'] = spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [6]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True, inferSchema=True)

In [7]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")


In [8]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

average_price_4_bedroom = spark.sql("""
    SELECT
        year(date) as Year,
        ROUND(AVG(price), 2) as Average_Price
    FROM
        home_sales
    WHERE
        bedrooms == 4
    GROUP BY
        year(date)
    ORDER BY
        Year
""")
average_price_4_bedroom.show()

+----+-------------+
|Year|Average_Price|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [11]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

average_price_3_bed_3_bath = spark.sql("""
    SELECT
        CAST(date_built AS STRING) as Year_Built,
        ROUND(AVG(price), 2) as Average_Price
    FROM
        home_sales
    WHERE
        bedrooms == 3 AND bathrooms == 3
    GROUP BY
        Year_Built
    ORDER BY
        Year_Built
""")
average_price_3_bed_3_bath.show()


+----------+-------------+
|Year_Built|Average_Price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [12]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

average_price_filtered = spark.sql("""
    SELECT
        CAST(date_built AS STRING) as Year_Built,
        ROUND(AVG(price), 2) as Average_Price
    FROM
        home_sales
    WHERE
        bedrooms == 3 AND
        bathrooms == 3 AND
        floors == 2 AND
        sqft_living >= 2000
    GROUP BY
        Year_Built
    ORDER BY
        Year_Built
""")
average_price_filtered.show()


+----------+-------------+
|Year_Built|Average_Price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [13]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

# Query to calculate the average price per view rating
average_price_per_view = spark.sql("""
    SELECT
        view AS View_Rating,
        ROUND(AVG(price), 2) AS Average_Price
    FROM
        home_sales
    GROUP BY
        view
    HAVING
        AVG(price) >= 350000
    ORDER BY
        View_Rating DESC
""")


print("--- %s seconds ---" % (time.time() - start_time))

--- 0.12192535400390625 seconds ---


In [14]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [17]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')


True

In [18]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

# Query to calculate the average price per view rating using cached data
average_price_per_view_cached = spark.sql("""
    SELECT
        view AS View_Rating,
        ROUND(AVG(price), 2) AS Average_Price
    FROM
        home_sales
    GROUP BY
        view
    HAVING
        AVG(price) >= 350000
    ORDER BY
        View_Rating DESC
""")

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.05103421211242676 seconds ---


In [19]:
# 10. Partition the data by "date_built" and write to a Parquet file
df.write.partitionBy("date_built").parquet("/content/home_sales_parquet")


In [20]:
# 11. Read the Parquet formatted data
parquet_df = spark.read.parquet("/content/home_sales_parquet")


In [21]:
# 12. Create a temporary table from the Parquet DataFrame
parquet_df.createOrReplaceTempView("parquet_home_sales")


In [22]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

# Query to calculate the average price per view rating using the Parquet data
average_price_per_view_parquet = spark.sql("""
    SELECT
        view AS View_Rating,
        ROUND(AVG(price), 2) AS Average_Price
    FROM
        parquet_home_sales
    GROUP BY
        view
    HAVING
        AVG(price) >= 350000
    ORDER BY
        View_Rating DESC
""")


print("--- %s seconds ---" % (time.time() - start_time))

--- 0.0701601505279541 seconds ---


In [23]:
# Uncache the home_sales temporary table
spark.sql("UNCACHE TABLE home_sales")

DataFrame[]